# Interaction Context

> Context management for interaction patterns providing access to state, request, and custom data

In [ ]:
#| default_exp core.context

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, Any, Optional
from dataclasses import dataclass, field

## InteractionContext Class

The `InteractionContext` class provides a unified interface for accessing state, request data, and custom data within interaction patterns. This allows render functions and handlers to access everything they need without passing many individual parameters.

In [ ]:
#| export
@dataclass
class InteractionContext:
    """Context for interaction patterns providing access to state, request, and custom data."""
    
    state: Dict[str, Any] = field(default_factory=dict)  # Workflow state
    request: Optional[Any] = None  # FastHTML request object
    session: Optional[Any] = None  # FastHTML session object
    data: Dict[str, Any] = field(default_factory=dict)  # Custom data from data loaders
    metadata: Dict[str, Any] = field(default_factory=dict)  # Additional metadata
    
    def get(self, 
            key: str,  # Key to retrieve from state
            default: Any = None  # Default value if key not found
           ) -> Any:  # Value from state or default
        """Get value from workflow state."""
        return self.state.get(key, default)
    
    def get_data(self, 
                 key: str,  # Key to retrieve from data
                 default: Any = None  # Default value if key not found
                ) -> Any:  # Value from data or default
        """Get value from custom data."""
        return self.data.get(key, default)
    
    def has(self, 
            key: str  # Key to check in state
           ) -> bool:  # True if key exists in state
        """Check if key exists in workflow state."""
        return key in self.state
    
    def set(self, 
            key: str,  # Key to set in state
            value: Any  # Value to store
           ) -> None:
        """Set value in workflow state."""
        self.state[key] = value
    
    def get_all_state(self) -> Dict[str, Any]:  # All workflow state
        """Get all workflow state as dictionary."""
        return self.state.copy()
    
    def update_state(self, 
                     updates: Dict[str, Any]  # State updates to apply
                    ) -> None:
        """Update multiple state values at once."""
        self.state.update(updates)

## Usage Examples

The `InteractionContext` is typically created and managed by interaction pattern classes (like `StepFlow`), but here are examples showing how it's used:

In [ ]:
# Create a context
ctx = InteractionContext()
ctx

InteractionContext(state={}, request=None, session=None, data={}, metadata={})

In [ ]:
# Store and retrieve state
ctx.set("plugin_id", "transcription_voxtral")
ctx.set("step", 1)

print(f"Plugin ID: {ctx.get('plugin_id')}")
print(f"Has file_path: {ctx.has('file_path')}")

Plugin ID: transcription_voxtral
Has file_path: False


In [ ]:
# Context with custom data (typically from data loaders)
ctx_with_data = InteractionContext(
    state={"user_id": "123"},
    data={
        "plugins": [
            {"name": "voxtral", "title": "Voxtral HF"},
            {"name": "whisper", "title": "Whisper"}
        ],
        "plugin_count": 5
    }
)

plugins = ctx_with_data.get_data("plugins", [])
print(f"Plugins available: {ctx_with_data.get_data('plugin_count')}")
print(f"First plugin: {plugins[0]}")

Plugins available: 5
First plugin: {'name': 'voxtral', 'title': 'Voxtral HF'}


In [ ]:
# Batch updates
ctx.update_state({
    "file_path": "/path/to/file.mp3",
    "confirmed": True
})

print(f"All state: {ctx.get_all_state()}")

All state: {'plugin_id': 'transcription_voxtral', 'step': 1, 'file_path': '/path/to/file.mp3', 'confirmed': True}


## Using Context in Render Functions

Here's how the context is typically used within step render functions:

In [ ]:
# Example render function signature
def render_plugin_selector(ctx: InteractionContext):
    """Render plugin selection step."""
    # Access loaded data
    plugins = ctx.get_data("plugins", [])
    
    # Access current state
    selected = ctx.get("plugin_id")
    
    # Could access request if needed
    # app_state = ctx.request.app.state
    
    # Return rendered UI
    return f"Render {len(plugins)} plugins, selected: {selected}"

In [ ]:
# Simulate using the render function
test_ctx = InteractionContext(
    state={"plugin_id": "plugin_1"},
    data={"plugins": [{}, {}]}  # 2 plugins
)

render_plugin_selector(test_ctx)

'Render 2 plugins, selected: plugin_1'

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()